In [5]:
import pandas as pd
df = pd.read_excel('talkdata.xlsx') 

In [6]:
def isEnglishOrKorean(input_s):
    k_count = 0
    e_count = 0
    for c in input_s:
        if ord('가') <= ord(c) <= ord('힣') or ord('ㄱ') <= ord(c) <= ord('ㅎ') or ord('ㅏ') <= ord(c) <= ord('ㅣ') :
            k_count+=1
        elif ord('a') <= ord(c.lower()) <= ord('z'):
            e_count+=1
    if k_count == 0 and e_count == 0:
        return "others"
    elif k_count>0:
        return "kor"
    else:
        return "eng"

In [7]:
#----------언어 Series 생성
lang = []
for i in df['plainText']:
    lang.append(isEnglishOrKorean(i))
df['language'] = pd.DataFrame(lang)

In [8]:
kor_chat = df[df['language']=='kor']

In [9]:
import re
import konlpy
from tqdm import tqdm
from konlpy.tag import Okt
from collections import Counter

In [10]:
text = kor_chat['plainText'].tolist()
print(len(text))

61937


In [11]:
# 한글/영어 표현만 남기기
def extract_word(text):
    hangul = re.compile('[^A-Za-z가-힣]')
    result = hangul.sub(' ', text)
    return result

In [12]:
kor_chat = kor_chat.reset_index(drop=True)
kor_chat.head(3)

,chatID,personType,personID,createdAt,plainText,language
0,Jac17549d6f7v,manager,T8758793187x,2021-01-04 10:32:35,안녕하세요 KULTIPS에 가이드 문서를 제공하겠습니다.,kor
1,Ub173640ed31u,manager,W6523714365l,2021-01-04 09:07:18,"기본적으로 영어로 코스가 생성된 경우, [Properties]에서 언어를 한국어로 ...",kor
2,Ub173640ed31u,user,N4714897d8f7t,2021-01-04 11:39:41,네.. 저도 불편하지만 학생들도 메뉴를 한눈에 알아보기 어려워서 어려움이 많아요. ...,kor


In [13]:
print("Before : ",kor_chat['plainText'][3])
print("After : ",extract_word(kor_chat['plainText'][3]))

Before :  네. 행복한 하루 보내세요😊
After :  네  행복한 하루 보내세요 


In [14]:
kor_chat['plainText'] = kor_chat['plainText'].apply(lambda x: extract_word(x))

In [15]:
# 형태소 분석
okt = Okt()
words = " ".join(kor_chat['plainText'].tolist())
words = okt.morphs(words,stem=True)

len(words)

878708

In [16]:
# 한 글자 단어(조사..) 제거
remove_one_word = [x for x in words if len(x)>1]
len(remove_one_word)

630758

In [17]:
from collections import Counter
frequent = Counter(remove_one_word).most_common()
frequent

[('하다', 34891),
 ('있다', 16616),
 ('되다', 13024),
 ('감사하다', 7898),
 ('으로', 7700),
 ('확인', 7288),
 ('교수', 6965),
 ('드리다', 6605),
 ('에서', 6185),
 ('않다', 5891),
 ('해주다', 5038),
 ('문의', 4831),
 ('블랙보드', 4634),
 ('같다', 4628),
 ('시간', 4549),
 ('학생', 4444),
 ('이다', 4347),
 ('강의', 4280),
 ('보다', 4274),
 ('출석', 4143),
 ('수업', 4027),
 ('없다', 3935),
 ('영상', 3734),
 ('되어다', 3633),
 ('답변', 3511),
 ('설정', 3355),
 ('하고', 3281),
 ('혹시', 3228),
 ('문제', 3058),
 ('내용', 3009),
 ('안녕하다', 2966),
 ('시험', 2870),
 ('어떻다', 2861),
 ('해보다', 2854),
 ('하루', 2802),
 ('경우', 2785),
 ('아니다', 2656),
 ('보내다', 2614),
 ('좋다', 2605),
 ('메뉴', 2590),
 ('다시', 2586),
 ('다른', 2576),
 ('코스', 2564),
 ('알다', 2501),
 ('뜨다', 2379),
 ('궁금하다', 2367),
 ('접속', 2352),
 ('께서', 2340),
 ('사용', 2285),
 ('편하다', 2230),
 ('가다', 2183),
 ('로그인', 2172),
 ('안되다', 2145),
 ('들어가다', 2109),
 ('조교', 2100),
 ('가능하다', 2099),
 ('보이다', 2086),
 ('남기다', 2067),
 ('종료', 2065),
 ('방법', 2015),
 ('해당', 2011),
 ('발생', 1985),
 ('그렇다', 1980),
 ('지금', 1939),
 ('등록', 1877

In [18]:
with open('stopwords.txt','r') as f:
    list_file = f.readlines()
stopwords = list_file[0].split(",")
remove_stopwords = [x for x in remove_one_word if x not in stopwords]
len(remove_stopwords)

441018

In [20]:
Counter(remove_stopwords).most_common()

[('감사하다', 7898),
 ('확인', 7288),
 ('교수', 6965),
 ('드리다', 6605),
 ('해주다', 5038),
 ('문의', 4831),
 ('블랙보드', 4634),
 ('학생', 4444),
 ('강의', 4280),
 ('출석', 4143),
 ('수업', 4027),
 ('영상', 3734),
 ('답변', 3511),
 ('설정', 3355),
 ('문제', 3058),
 ('내용', 3009),
 ('안녕하다', 2966),
 ('시험', 2870),
 ('해보다', 2854),
 ('하루', 2802),
 ('경우', 2785),
 ('보내다', 2614),
 ('좋다', 2605),
 ('메뉴', 2590),
 ('코스', 2564),
 ('뜨다', 2379),
 ('궁금하다', 2367),
 ('접속', 2352),
 ('께서', 2340),
 ('사용', 2285),
 ('편하다', 2230),
 ('로그인', 2172),
 ('들어가다', 2109),
 ('조교', 2100),
 ('가능하다', 2099),
 ('남기다', 2067),
 ('종료', 2065),
 ('방법', 2015),
 ('해당', 2011),
 ('발생', 1985),
 ('등록', 1877),
 ('맞다', 1863),
 ('클릭', 1837),
 ('두시', 1832),
 ('화면', 1814),
 ('제출', 1814),
 ('부탁드리다', 1790),
 ('녹화', 1759),
 ('누르다', 1759),
 ('많다', 1747),
 ('과제', 1741),
 ('브라우저', 1734),
 ('삭제', 1657),
 ('걸리다', 1598),
 ('사항', 1570),
 ('현재', 1561),
 ('말씀', 1494),
 ('관련', 1452),
 ('파일', 1420),
 ('락다운', 1416),
 ('기록', 1399),
 ('버튼', 1359),
 ('변경', 1341),
 ('체크', 1340),
 ('바라다', 1337

In [21]:
# 최소 횟수 이하 단어 제거
for item, count in Counter(remove_stopwords).most_common():
    if count==1:
        print(item)

Properties
Application
인생
년월
재체점
사건
김재일
이민영
분제
스탬프
주체
타임스탬프
단서
onedrive
대별
원드라이브
폴더탭
bar
생물통계학
BIOSTATISTICS
강다현
zomm
원격확
암벽등반
JEPG
International
business
안고
유뷰
브로
이루
심의
뒤즘
무난
가상메모리
fit
confluence
uconn
ikb
huskyct
troubleshooting
낼때
말순
마이너스
Assinment
tap
Retired
연한
ryanchoi
만으로도
다봣는데
셔서
증원
홍지호
포스터
간접
주진혁
말기
hjyu
박유진
PKIX
path
validation
java
security
cert
CertPathValidatorException
validity
명절
박상
폭력
anamouheung
이지영
교육인
반디캠
균등하다
표지
QWICKLY
빠빠이
한국인
우싱
뮬리
만화
aff
cafa
이차전지
고지서
장기간
디다
suminanny
praveen
부임
정연
firstjoin
lastleave
대세
정수연
chefcurry
CIE
Not
지적재산
부모
콜렙
순수
제품디자인
성업
유용
KUKIST
notfound
Id
발습
대학생
입학처
정우준
미정
사이버
궁금힙니
어딨다
일산
물릿
Recipients
음영
CC
BCC
동이
FAIpQLSd
AiCZugLtB
mppSU
oETRtOuLvfc
BGbsda
에스더
간곡히
CB
통치
PRESIDENTIAL
SYSTEM
미학
송희경입니
벌어지다
상강
PASS
학과목
HTTP
Status
Bad
Request
글시
다형
Xs
영국
이주환
중임
토픽
우선권
real
쪼개다
이민지
대치
쿠타
국어생활
anouncement
statistic
수험
bcc
Have
김성진
박재연
최수현
요셉
이문
강위
신호처리
김태곤
신지윤
형사법
mincrosft
힌대
동하
못내
될떄
삼국유사
code
technipages
troubleshoot
Black
도데
쭈르륵
이규성
유니코드
김해
SEMICON

In [25]:
# 3번 이상 나온 단어만 추출 (개수는 논의 필요)
minimum_count = 3
more_than_one_time = []
for i in tqdm(range(len(remove_stopwords))):
    tmp = remove_stopwords[i]
    if remove_stopwords.count(tmp) >= minimum_count:
        more_than_one_time.append(tmp)

100%|███████████████████████████████████████████████████████████████████████| 441018/441018 [16:26:44<00:00,  7.45it/s]


In [37]:
word_list = Counter(more_than_one_time).most_common()
word_list

[('감사하다', 7898),
 ('확인', 7288),
 ('교수', 6965),
 ('드리다', 6605),
 ('해주다', 5038),
 ('문의', 4831),
 ('블랙보드', 4634),
 ('학생', 4444),
 ('강의', 4280),
 ('출석', 4143),
 ('수업', 4027),
 ('영상', 3734),
 ('답변', 3511),
 ('설정', 3355),
 ('문제', 3058),
 ('내용', 3009),
 ('안녕하다', 2966),
 ('시험', 2870),
 ('해보다', 2854),
 ('하루', 2802),
 ('경우', 2785),
 ('보내다', 2614),
 ('좋다', 2605),
 ('메뉴', 2590),
 ('코스', 2564),
 ('뜨다', 2379),
 ('궁금하다', 2367),
 ('접속', 2352),
 ('께서', 2340),
 ('사용', 2285),
 ('편하다', 2230),
 ('로그인', 2172),
 ('들어가다', 2109),
 ('조교', 2100),
 ('가능하다', 2099),
 ('남기다', 2067),
 ('종료', 2065),
 ('방법', 2015),
 ('해당', 2011),
 ('발생', 1985),
 ('등록', 1877),
 ('맞다', 1863),
 ('클릭', 1837),
 ('두시', 1832),
 ('화면', 1814),
 ('제출', 1814),
 ('부탁드리다', 1790),
 ('녹화', 1759),
 ('누르다', 1759),
 ('많다', 1747),
 ('과제', 1741),
 ('브라우저', 1734),
 ('삭제', 1657),
 ('걸리다', 1598),
 ('사항', 1570),
 ('현재', 1561),
 ('말씀', 1494),
 ('관련', 1452),
 ('파일', 1420),
 ('락다운', 1416),
 ('기록', 1399),
 ('버튼', 1359),
 ('변경', 1341),
 ('체크', 1340),
 ('바라다', 1337

In [38]:
import csv
with open ("word_list2.csv","w",newline='',encoding='euc-kr') as f:
    csvw = csv.writer(f)
    for v in word_list:
        csvw.writerow(v)